In [3]:
# -*- coding: utf-8 -*-
#!/usr/bin/env python3
"""
Generate realistic MailMkt SQLite databases, using the original table names.
- email_map.db: table `email_map` with columns 
  (campaign_id, msg_id, recipient, variant, send_ts)
- email_events.db: table `event_log`
- campaigns.db: tables `campaigns` and `user_signup`
"""

import sqlite3       # Módulo para interactuar con SQLite
import uuid          # Para generar IDs únicos (UUID4)
import random        # Para generar valores aleatorios
from datetime import datetime, timedelta  # Para manejar fechas y rangos de tiempo
from typing import List, Dict           # Para anotaciones de tipo
import pandas as pd   # Para trabajar con DataFrames

# Constants
NUM_CAMPAIGNS = 10                          # Número de campañas a generar
NUM_CLIENTS = 20                            # Número de clientes únicos
CLIENTS: List[str] = [                      # Lista de nombres de cliente: "client1", ..., "client20"
    f"client{i}" for i in range(1, NUM_CLIENTS + 1)
]
EMAIL_TEMPLATE = "default.tester.mail+{client}@gmail.com"  # Plantilla de email con placeholder
NOW = datetime.utcnow()         # Timestamp actual en UTC para referenciar ventanas de tiempo


def generate_campaigns() -> pd.DataFrame:
    """
    Genera un DataFrame de campañas con:
      - campaign_id: UUID
      - name: "campaign i"
      - start_date: fecha de inicio aleatoria entre 6 meses y 3 semanas atrás
      - end_date: fecha de fin aleatoria entre start_date+3 semanas y NOW
      - budget: entero aleatorio entre 1,000 y 50,000
    """
    campaigns = []  # Lista para acumular diccionarios de campaña
    for i in range(1, NUM_CAMPAIGNS + 1):
        cid = str(uuid.uuid4())  # Genera un UUID único y lo convierte a string
        name = f"campaign {i}"   # Nombre legible de la campaña
        # Define ventana de posibles fechas de inicio
        start_earliest = NOW - timedelta(days=180)  # Hace 6 meses
        start_latest = NOW - timedelta(days=21)     # Hace 3 semanas
        # Fecha de inicio aleatoria dentro de la ventana
        start_date = start_earliest + (
            start_latest - start_earliest
        ) * random.random()
        # Duración máxima = hasta NOW, mínima = 21 días
        max_dur = min(timedelta(days=180), NOW - start_date)
        min_dur = timedelta(days=21)
        # Duración aleatoria entre min_dur y max_dur
        duration = min_dur + (max_dur - min_dur) * random.random()
        end_date = start_date + duration  # Fecha de fin sumando duración
        budget = random.randint(1_000, 50_000)  # Presupuesto aleatorio
        # Añade diccionario de esta campaña a la lista
        campaigns.append({
            "campaign_id": cid,
            "name": name,
            "start_date": start_date,
            "end_date": end_date,
            "budget": budget,
        })
    # Convierte la lista de diccionarios en un DataFrame de pandas
    return pd.DataFrame(campaigns)


def assign_qualities(camps: pd.DataFrame) -> Dict[str, Dict[str, float]]:
    """
    Asigna probabilidades de comportamiento por campaña:
      - 'p_open', 'p_click', 'p_unsub', 'p_comp', 'p_signup'
    Diferencia campaña 1 (muy buena), campaña 2 (muy mala) y el resto.
    """
    qualities: Dict[str, Dict[str, float]] = {}
    for _, row in camps.iterrows():
        cid, name = row["campaign_id"], row["name"]
        if name == "campaign 1":
            # Campaña muy buena: altas tasas
            qualities[cid] = {
                "p_open": random.uniform(0.5, 0.8),
                "p_click": random.uniform(0.25, 0.4),
                "p_unsub": random.uniform(0.01, 0.03),
                "p_comp": random.uniform(0.0, 0.01),
                "p_signup": random.uniform(0.10, 0.2),
            }
        elif name == "campaign 2":
            # Campaña muy mala: bajas tasas
            qualities[cid] = {
                "p_open": random.uniform(0.01, 0.1),
                "p_click": random.uniform(0.0, 0.02),
                "p_unsub": random.uniform(0.01, 0.05),
                "p_comp": random.uniform(0.0, 0.01),
                "p_signup": random.uniform(0.01, 0.03),
            }
        else:
            # Resto de campañas: tasas intermedias
            qualities[cid] = {
                "p_open": random.uniform(0.1, 0.5),
                "p_click": random.uniform(0.02, 0.25),
                "p_unsub": random.uniform(0.01, 0.05),
                "p_comp": random.uniform(0.0, 0.01),
                "p_signup": random.uniform(0.03, 0.1),
            }
    return qualities  # Diccionario {campaign_id: quality_params}


def generate_send_log(camps: pd.DataFrame) -> pd.DataFrame:
    """
    Genera un DataFrame 'email_map' con envíos:
      - campaign_id, msg_id, recipient, variant, send_ts
    variant: A/B aleatoria
    send_ts: fecha de envío aleatoria entre start y end de la campaña
    """
    records = []  # Lista de registros
    for _, row in camps.iterrows():
        cid = row["campaign_id"]
        # Itera sobre cada cliente
        for client in CLIENTS:
            records.append({
                #"campaign_id": cid,
                "msg_id": str(uuid.uuid4()),  # ID único por mensaje
                "recipient": EMAIL_TEMPLATE.format(client=client),
                #"variant": random.choice(["A", "B"]),  # Test A/B
                "send_ts": row["start_date"]
                           + (row["end_date"] - row["start_date"])
                             * random.random()
            })
    return pd.DataFrame(records)  # Devuelve DataFrame de envíos


def generate_event_log(
    sends: pd.DataFrame,
    qualities: Dict[str, dict],
    camps: pd.DataFrame
) -> pd.DataFrame:
    """
    Simula los eventos en email_events.db:
      - always 'send'
      - 'open' con probabilidad p_open
      - si abre, posible 'click' con p_click
      - si no abre, 'deleted_or_spam' tras 3 días
      - 'unsubscribe' y 'complaint' según probabilidad
    """
    # Mapa rápido de metadatos de campaña
    camp_map = camps.set_index("campaign_id").to_dict("index")
    records = []
    # Recorre cada fila de send_df
    for _, send in sends.iterrows():
        cid, msg, ts = send["campaign_id"], send["msg_id"], send["send_ts"]
        q = qualities[cid]  # Parametros de calidad para esta campaña
        # Evento 'send'
        records.append((cid, msg, "send", ts))
        # Decidir si 'open' o 'deleted_or_spam'
        if random.random() < q["p_open"]:
            # Si abre, timestamp entre 0.5 y 72 horas después
            o_ts = ts + timedelta(hours=random.uniform(0.5, 72))
            records.append((cid, msg, "open", o_ts))
            # Posible 'click'
            if random.random() < q["p_click"]:
                c_ts = o_ts + timedelta(hours=random.uniform(0.1, 48))
                records.append((cid, msg, "click", c_ts))
        else:
            # Si no abre en 3 días → 'deleted_or_spam'
            dos_ts = ts + timedelta(days=3, hours=random.uniform(0, 12))
            records.append((cid, msg, "deleted_or_spam", dos_ts))
        # Posible 'unsubscribe'
        span_days = max(1, (camp_map[cid]["end_date"] - ts).days)
        if random.random() < q["p_unsub"]:
            u_ts = ts + timedelta(days=random.uniform(1, span_days))
            records.append((cid, msg, "unsubscribe", u_ts))
        # Posible 'complaint'
        if random.random() < q["p_comp"]:
            comp_ts = ts + timedelta(days=random.uniform(1, span_days))
            records.append((cid, msg, "complaint", comp_ts))
    # Convierte lista de tuplas en DataFrame con columnas fijas
    return pd.DataFrame(records, columns=[
        "campaign_id", "msg_id", "event_type", "event_ts"
    ])


def generate_user_signups(
    camps: pd.DataFrame,
    qualities: Dict[str, dict]
) -> pd.DataFrame:
    """
    Simula la tabla user_signup:
      - según p_signup, número de clientes que se dan de alta
      - signup_ts entre start_date y end_date
    """
    records = []
    for _, row in camps.iterrows():
        cid = row["campaign_id"]
        p = qualities[cid]["p_signup"]
        n = max(1, int(NUM_CLIENTS * p))  # Al menos un signup
        sampled = random.sample(CLIENTS, n)  # Clientes muestreados
        for client in sampled:
            records.append({
                "signup_id": str(uuid.uuid4()),
                "campaign_id": cid,
                "client_name": client,
                "email": EMAIL_TEMPLATE.format(client=client),
                "signup_ts": row["start_date"]
                             + timedelta(days=random.uniform(
                                1,
                                max(1, (row["end_date"]
                                        - row["start_date"]).days)
                             ))
            })
    return pd.DataFrame(records)


def write_db(path: str, table: str, df: pd.DataFrame, schema_sql: str) -> None:
    """
    Escribe un DataFrame en un archivo SQLite:
      - borra la tabla si existe
      - ejecuta el schema_sql para crear la tabla
      - inserta los datos del DataFrame
    """
    conn = sqlite3.connect(path, detect_types=sqlite3.PARSE_DECLTYPES)
    # DROP y CREATE
    conn.executescript(f"DROP TABLE IF EXISTS {table};\n{schema_sql}")
    # Inserta DataFrame
    df.to_sql(table, conn, if_exists="append", index=False)
    conn.close()


# ---------- EJECUCIÓN PRINCIPAL ----------

# 1) Genera DataFrames
campaigns_df = generate_campaigns()                   # Campañas
qualities = assign_qualities(campaigns_df)            # Probabilidades de rendimiento
send_df = generate_send_log(campaigns_df)             # Registros de envío
event_df = generate_event_log(send_df, qualities, campaigns_df)  # Eventos simulados
signup_df = generate_user_signups(campaigns_df, qualities)       # Altas de usuarios

# 2) Define esquemas SQL idénticos a tablas originales
email_map_schema = """
CREATE TABLE email_map(
    campaign_id TEXT,
    msg_id       TEXT,
    recipient    TEXT,
    variant      TEXT,
    send_ts      TIMESTAMP
);
"""
event_schema = """
CREATE TABLE event_log(
    campaign_id TEXT,
    msg_id       TEXT,
    event_type   TEXT,
    event_ts     TIMESTAMP
);
"""
campaigns_schema = """
CREATE TABLE campaigns(
    campaign_id TEXT PRIMARY KEY,
    name         TEXT,
    start_date   TIMESTAMP,
    end_date     TIMESTAMP,
    budget       REAL
);
"""
signup_schema = """
CREATE TABLE user_signup(
    signup_id   TEXT PRIMARY KEY,
    campaign_id TEXT,
    client_name TEXT,
    email       TEXT,
    signup_ts   TIMESTAMP
);
"""

# 3) Graba en archivos .db
write_db("email_map.db",    "email_map",    send_df,     email_map_schema)
write_db("email_events.db", "event_log",    event_df,    event_schema)
write_db("campaigns.db",    "campaigns",    campaigns_df, campaigns_schema)
write_db("campaigns.db",    "user_signup",  signup_df,    signup_schema)

print("Databases created with schemas identical to originals plus campaign_id added to email_map.")





C:\Users\jvvce\AppData\Local\Temp\ipykernel_15840\2613396930.py:25: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  NOW = datetime.utcnow()         # Timestamp actual en UTC para referenciar ventanas de tiempo


NameError: name 'cid' is not defined

KeyboardInterrupt: 

In [ ]:
# Adjusted DB Generation: create `events` table for compatibility

import sqlite3

# After generating email_events.db with `event_log` table...

conn_ev = sqlite3.connect("email_events.db", detect_types=sqlite3.PARSE_DECLTYPES)
# Drop existing compatibility table if any
conn_ev.execute("DROP TABLE IF EXISTS events;")
# Create `events` table matching the app's expected schema:
conn_ev.execute("""
CREATE TABLE events AS
SELECT
    msg_id,
    event_type,
    NULL AS client_ip,       -- placeholder (no IP simulated)
    event_ts AS ts,          -- rename event_ts to ts
    campaign_id AS campaign  -- rename campaign_id to campaign
FROM event_log;
""")
conn_ev.commit()
conn_ev.close()

print("Created compatibility table `events` in email_events.db")


In [ ]:
#!/usr/bin/env python3
"""
Migra campaigns.db para añadir start_date y end_date
a la tabla campaigns, extrayéndolas de email_events.db.
"""

import sqlite3

CAMPAIGNS_DB = "campaigns.db"
EVENTS_DB    = "email_events.db"

def migrate_campaigns_dates(camp_db: str, events_db: str) -> None:
    conn = sqlite3.connect(camp_db, detect_types=sqlite3.PARSE_DECLTYPES)
    cur = conn.cursor()

    # Adjunta la BD de eventos
    cur.execute(f"ATTACH DATABASE '{events_db}' AS evdb;")

    # 1) Añadir columnas si no existen
    cur.execute("PRAGMA table_info(campaigns);")
    cols = [row[1] for row in cur.fetchall()]
    if "start_date" not in cols:
        cur.execute("ALTER TABLE campaigns ADD COLUMN start_date TIMESTAMP;")
    if "end_date" not in cols:
        cur.execute("ALTER TABLE campaigns ADD COLUMN end_date TIMESTAMP;")

    # 2) Rellenar start_date y end_date a partir de evdb.event_log
    cur.execute("""
    UPDATE campaigns
    SET
      start_date = (
        SELECT MIN(event_ts)
        FROM evdb.event_log
        WHERE evdb.event_log.campaign_id = campaigns.campaign_id
      ),
      end_date   = (
        SELECT MAX(event_ts)
        FROM evdb.event_log
        WHERE evdb.event_log.campaign_id = campaigns.campaign_id
      )
    ;
    """)

    conn.commit()
    conn.close()
    print("Migración completada: start_date y end_date actualizados en campaigns.db")

if __name__ == "__main__":
    migrate_campaigns_dates(CAMPAIGNS_DB, EVENTS_DB)



In [1]:
#!/usr/bin/env python3
"""
Re-generate the `events` compatibility table in email_events.db so that
the `campaign` column holds the campaign *name* instead of the ID.
"""

import sqlite3

CAMPAIGNS_DB = "campaigns.db"
EVENTS_DB    = "email_events.db"

def rebuild_events_with_names(events_db: str, campaigns_db: str) -> None:
    # Connect to email_events.db
    conn = sqlite3.connect(events_db, detect_types=sqlite3.PARSE_DECLTYPES)
    cur = conn.cursor()

    # Attach campaigns.db to pull campaign names
    cur.execute(f"ATTACH DATABASE '{campaigns_db}' AS cdb;")

    # Drop old compatibility table if exists
    cur.execute("DROP TABLE IF EXISTS events;")

    # Re-create `events` table, joining to get campaign name
    cur.execute("""
    CREATE TABLE events AS
    SELECT
      e.msg_id,
      e.event_type,
      NULL      AS client_ip,
      e.event_ts AS ts,
      cdb.campaigns.name AS campaign
    FROM event_log AS e
    JOIN cdb.campaigns
      ON e.campaign_id = cdb.campaigns.campaign_id
    ;
    """)

    conn.commit()
    conn.close()
    print("Rebuilt `events` table: now `campaign` column contains campaign names.")

if __name__ == "__main__":
    rebuild_events_with_names(EVENTS_DB, CAMPAIGNS_DB)


Rebuilt `events` table: now `campaign` column contains campaign names.


In [1]:
#!/usr/bin/env python3
"""
Genera tres bases de datos SQLite idénticas en esquema a las tuyas:

1) email_map.db  → table email_map(campaign_id, msg_id, recipient, variant, send_ts)
2) email_events.db
     • event_log(campaign_id, msg_id, event_type, event_ts)
     • events(msg_id, event_type, client_ip, ts, campaign)
3) campaigns.db
     • campaigns(campaign_id, name, start_date, end_date, budget)
     • user_signup(signup_id, campaign_id, client_name, email, signup_ts)
"""

import sqlite3
import uuid
import random
from datetime import datetime, timedelta
import pandas as pd

# ─── Parámetros ───────────────────────────────────────────
NUM_CAMPAIGNS = 10
NUM_CLIENTS   = 20
CLIENTS       = [f"client{i}" for i in range(1, NUM_CLIENTS + 1)]
EMAIL_TMPL    = "default.tester.mail+{client}@gmail.com"
NOW           = datetime.utcnow()

# ─── Helpers ──────────────────────────────────────────────
def random_ipv4() -> str:
    return ".".join(str(random.randint(1, 254)) for _ in range(4))

def generate_campaigns() -> pd.DataFrame:
    rows = []
    for i in range(1, NUM_CAMPAIGNS + 1):
        cid = str(uuid.uuid4())
        name = f"campaign {i}"
        # rango de inicio entre hace 6 meses y hace 3 semanas
        start_earliest = NOW - timedelta(days=180)
        start_latest   = NOW - timedelta(days=21)
        start_date     = start_earliest + (start_latest - start_earliest) * random.random()
        # duracion entre 21 y 180 días
        max_dur   = min(timedelta(days=180), NOW - start_date)
        min_dur   = timedelta(days=21)
        duration  = min_dur + (max_dur - min_dur) * random.random()
        end_date  = start_date + duration
        budget    = random.randint(1_000, 50_000)
        rows.append({
            "campaign_id": cid,
            "name": name,
            "start_date": start_date,
            "end_date": end_date,
            "budget": budget,
        })
    return pd.DataFrame(rows)

def assign_qualities(camps: pd.DataFrame) -> dict:
    Q = {}
    for _, r in camps.iterrows():
        cid, name = r["campaign_id"], r["name"]
        if name == "campaign 1":       # muy buena
            Q[cid] = dict(
                p_open= random.uniform(0.5, 0.8),
                p_click=random.uniform(0.25,0.4),
                p_unsub=random.uniform(0.01,0.03),
                p_comp= random.uniform(0.0,0.01),
                p_signup=random.uniform(0.10,0.2),
            )
        elif name == "campaign 2":     # muy mala
            Q[cid] = dict(
                p_open= random.uniform(0.01,0.1),
                p_click=random.uniform(0.0,0.02),
                p_unsub=random.uniform(0.01,0.05),
                p_comp= random.uniform(0.0,0.01),
                p_signup=random.uniform(0.01,0.03),
            )
        else:                          # intermedias
            Q[cid] = dict(
                p_open= random.uniform(0.1,0.5),
                p_click=random.uniform(0.02,0.25),
                p_unsub=random.uniform(0.01,0.05),
                p_comp= random.uniform(0.0,0.01),
                p_signup=random.uniform(0.03,0.1),
            )
    return Q

def generate_send_log(camps: pd.DataFrame) -> pd.DataFrame:
    recs = []
    for _, c in camps.iterrows():
        cid = c["campaign_id"]
        for client in CLIENTS:
            send_ts = c["start_date"] + (c["end_date"] - c["start_date"]) * random.random()
            recs.append({
                "campaign_id": cid,
                "msg_id": str(uuid.uuid4()),
                "recipient": EMAIL_TMPL.format(client=client),
                "variant": random.choice(["A","B"]),
                "send_ts": send_ts,
            })
    return pd.DataFrame(recs)

def generate_event_log(sends: pd.DataFrame, Q: dict, camps: pd.DataFrame) -> pd.DataFrame:
    camp_info = camps.set_index("campaign_id").to_dict("index")
    recs = []
    for _, s in sends.iterrows():
        cid, msg, ts = s["campaign_id"], s["msg_id"], s["send_ts"]
        q = Q[cid]
        # send
        recs.append((cid,msg,"send",ts))
        # open / deleted_or_spam
        if random.random() < q["p_open"]:
            ot = ts + timedelta(hours=random.uniform(0.5,72))
            recs.append((cid,msg,"open",ot))
            if random.random() < q["p_click"]:
                ct = ot + timedelta(hours=random.uniform(0.1,48))
                recs.append((cid,msg,"click",ct))
        else:
            dt = ts + timedelta(days=3, hours=random.uniform(0,12))
            recs.append((cid,msg,"deleted_or_spam",dt))
        # unsubscribe
        span = max(1, (camp_info[cid]["end_date"]-ts).days)
        if random.random() < q["p_unsub"]:
            ut = ts + timedelta(days=random.uniform(1,span))
            recs.append((cid,msg,"unsubscribe",ut))
        # complaint
        if random.random() < q["p_comp"]:
            ct = ts + timedelta(days=random.uniform(1,span))
            recs.append((cid,msg,"complaint",ct))
    return pd.DataFrame(recs, columns=["campaign_id","msg_id","event_type","event_ts"])

def generate_user_signup(camps: pd.DataFrame, Q: dict) -> pd.DataFrame:
    recs = []
    for _, c in camps.iterrows():
        cid = c["campaign_id"]
        n = max(1, int(NUM_CLIENTS * Q[cid]["p_signup"]))
        for client in random.sample(CLIENTS, n):
            st = c["start_date"] + timedelta(days=random.uniform(1,(c["end_date"]-c["start_date"]).days))
            recs.append({
                "signup_id": str(uuid.uuid4()),
                "campaign_id": cid,
                "client_name": client,
                "email": EMAIL_TMPL.format(client=client),
                "signup_ts": st,
            })
    return pd.DataFrame(recs)

# ─── Esquemas ─────────────────────────────────────────────
SCHEMA_EMAIL_MAP = """
CREATE TABLE email_map(
  campaign_id TEXT,
  msg_id       TEXT,
  recipient    TEXT,
  variant      TEXT,
  send_ts      TIMESTAMP
);
"""

SCHEMA_EVENT_LOG = """
CREATE TABLE event_log(
  campaign_id TEXT,
  msg_id       TEXT,
  event_type   TEXT,
  event_ts     TIMESTAMP
);
"""

SCHEMA_EVENTS = """
CREATE TABLE events(
  msg_id     TEXT,
  event_type TEXT,
  client_ip  TEXT,
  ts         TIMESTAMP,
  campaign   TEXT
);
"""

SCHEMA_CAMPAIGNS = """
CREATE TABLE campaigns(
  campaign_id TEXT PRIMARY KEY,
  name        TEXT,
  start_date  TIMESTAMP,
  end_date    TIMESTAMP,
  budget      REAL
);
"""

SCHEMA_SIGNUP = """
CREATE TABLE user_signup(
  signup_id   TEXT PRIMARY KEY,
  campaign_id TEXT,
  client_name TEXT,
  email       TEXT,
  signup_ts   TIMESTAMP
);
"""

# ─── Escritura de cada DB ──────────────────────────────────
def write_sqlite(path: str, table: str, df: pd.DataFrame, schema: str):
    conn = sqlite3.connect(path, detect_types=sqlite3.PARSE_DECLTYPES)
    cur  = conn.cursor()
    cur.executescript(f"DROP TABLE IF EXISTS {table};\n{schema}")
    df.to_sql(table, conn, if_exists="append", index=False)
    conn.close()

def build_all():
    # 1) campaigns.db
    camps_df = generate_campaigns()
    qualities= assign_qualities(camps_df)
    signup_df= generate_user_signup(camps_df, qualities)
    write_sqlite("campaigns.db", "campaigns",   camps_df, SCHEMA_CAMPAIGNS)
    write_sqlite("campaigns.db", "user_signup", signup_df, SCHEMA_SIGNUP)

    # 2) email_map.db
    sends_df = generate_send_log(camps_df)
    write_sqlite("email_map.db", "email_map", sends_df, SCHEMA_EMAIL_MAP)

    # 3) email_events.db → event_log + events
    ev_df    = generate_event_log(sends_df, qualities, camps_df)
    write_sqlite("email_events.db", "event_log", ev_df, SCHEMA_EVENT_LOG)

    # Construimos `events` uniendo nombre de campaña y añadiendo IP
    # Cargamos event_log + campaigns
    conn_e = sqlite3.connect("email_events.db", detect_types=sqlite3.PARSE_DECLTYPES)
    elog   = pd.read_sql("SELECT * FROM event_log", conn_e)
    camps  = pd.read_sql("SELECT campaign_id, name FROM campaigns", sqlite3.connect("campaigns.db"))
    conn_e.close()
    evt = elog.merge(camps, on="campaign_id", how="left")
    evt["client_ip"] = evt.apply(lambda _: random_ipv4(), axis=1)
    evt = evt.rename(columns={"event_ts":"ts", "name":"campaign"})
    evt = evt[["msg_id","event_type","client_ip","ts","campaign"]]
    write_sqlite("email_events.db", "events", evt, SCHEMA_EVENTS)

    print("▶️ Todas las bases de datos creadas con el esquema exacto original + nuevos campos.")

if __name__ == "__main__":
    build_all()


▶️ Todas las bases de datos creadas con el esquema exacto original + nuevos campos.


C:\Users\jvvce\AppData\Local\Temp\ipykernel_20500\549666373.py:25: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  NOW           = datetime.utcnow()


In [ ]:
#!/usr/bin/env python3
"""
Generate three SQLite databases (same schema as the original) but with
campaigns tied to MauBank product names and realistic email funnel metrics.

Databases and tables:
1) email_map.db
   - email_map(campaign_id, msg_id, recipient, variant, send_ts)

2) email_events.db
   - event_log(campaign_id, msg_id, event_type, event_ts)
   - events(msg_id, event_type, client_ip, ts, campaign)

3) campaigns.db
   - campaigns(campaign_id, name, start_date, end_date, budget)
   - user_signup(signup_id, campaign_id, client_name, email, signup_ts)
"""

from __future__ import annotations

import random
import sqlite3
import uuid
from datetime import datetime, timedelta
from typing import Dict, Iterable, List, Literal, Tuple

import pandas as pd

# ──────────────────────────────────────────────────────────────
# Configuration
# ──────────────────────────────────────────────────────────────

RANDOM_SEED: int = 12345
random.seed(RANDOM_SEED)

# Base audience
NUM_CLIENTS: int = 1000
CLIENTS: List[str] = [f"client{i}" for i in range(1, NUM_CLIENTS + 1)]
EMAIL_TMPL: str = "default.tester.mail+{client}@gmail.com"
NOW: datetime = datetime.utcnow()

# Products provided by the user (deduplicated, normalized)
RAW_PRODUCTS: List[str] = [
    "Savings Account",
    "Savings Account with Cheque Book",
    "Junior Savings Account",
    "Senior Prime Account",
    "MoneyGrow Savings Account",
    "Foreign Currency Account",
    "Current Account",
    "Term Deposit Account",
    "Fixed Deposit",
    "Personal Loan",
    "Home Loan",
    "Car Loan",
    "Education Loan",
    "Zero-Interest Medical Loan Scheme",
    "Green Financing Scheme",
    "Solar Panel Loan (Individuals)",
    "Solar Water Heater Loan (Individuals)",
    "Solar Panel Loan (Businesses)",
    "Solar Water Heater Loan (Businesses)",
    "Vehicle Leasing",
    "Medical Equipment Leasing",
    "Visa Credit Card Classic",
    "Visa Credit Card Gold",
    "Visa Credit Card Platinum",
    "Visa Debit Card (Classic)",
    "Visa Debit Platinum",
    "SME Classic Current Account",
    "SME Foreign Currency Account",
    "International Multi-Currency Account",
    "International Term Deposit Account",
    "International Loans and Overdrafts",
    "Business Loan",
    "Overdraft",
    "Foreign Currency Loan",
    "Project Finance",
    "Equipment Leasing",
    "Letter of Credit",
    "Bank Guarantees",
    "Spot FX",
    "Forward FX",
    "FX Swaps",
    "SWIFT International Transfers",
    "Bank Drafts",
    "Remittances",
    "Standing Orders",
    "Direct Debits",
    "e-Statements",
    #"SMS/Email Alerts",
    "Internet Banking",
    "MauBank WithMe (Mobile App)",
    "Travel Insurance",
    "Home Insurance",
    "Personal Accident Insurance",
    "Marine Insurance",
    "Motor Vehicle Insurance",
    "Private Banking",
]

# Ensure generic categories are present
EXTRA_GENERIC: List[str] = ["Loans", "Mortgages", "Credit Cards"]

# Optional: limit total campaigns (None = one per product)
# If you set MAX_CAMPAIGNS, products are sampled.
MAX_CAMPAIGNS: int | None = None  # e.g., 60 or None

# Campaign timing & budget ranges
START_RANGE_DAYS: Tuple[int, int] = (1085, 21)   # between 6 months ago and 3 weeks ago
MIN_DURATION_DAYS: int = 21
MAX_DURATION_DAYS: int = 360

# ──────────────────────────────────────────────────────────────
# Schemas (unchanged)
# ──────────────────────────────────────────────────────────────

SCHEMA_EMAIL_MAP: str = """
CREATE TABLE email_map(
  campaign_id TEXT,
  msg_id       TEXT,
  recipient    TEXT,
  variant      TEXT,
  send_ts      TIMESTAMP
);
"""

SCHEMA_EVENT_LOG: str = """
CREATE TABLE event_log(
  campaign_id TEXT,
  msg_id       TEXT,
  event_type   TEXT,
  event_ts     TIMESTAMP
);
"""

SCHEMA_EVENTS: str = """
CREATE TABLE events(
  msg_id     TEXT,
  event_type TEXT,
  client_ip  TEXT,
  ts         TIMESTAMP,
  campaign   TEXT
);
"""

SCHEMA_CAMPAIGNS: str = """
CREATE TABLE campaigns(
  campaign_id TEXT PRIMARY KEY,
  name        TEXT,
  start_date  TIMESTAMP,
  end_date    TIMESTAMP,
  budget      REAL
);
"""

SCHEMA_SIGNUP: str = """
CREATE TABLE user_signup(
  signup_id   TEXT PRIMARY KEY,
  campaign_id TEXT,
  client_name TEXT,
  email       TEXT,
  signup_ts   TIMESTAMP
);
"""

# ──────────────────────────────────────────────────────────────
# Taxonomy & baselines
# ──────────────────────────────────────────────────────────────

Category = Literal[
    "Retail",
    "Deposits",
    "Loans",
    "Mortgages",
    "Cards",
    "Leasing",
    "SME",
    "TradeFinance",
    "TreasuryFX",
    "International",
    "PaymentsOps",
    "Digital",
    "Insurance",
    "PrivateBanking",
]

# Simple classifier by keyword (order matters)
CATEGORY_RULES: List[Tuple[Category, Tuple[str, ...]]] = [
    ("Mortgages", ("mortgage", "home loan", "housing loan")),
    ("Loans", ("loan", "loans")),  # generic
    ("Cards", ("credit card", "debit")),
    ("Leasing", ("leasing",)),
    ("TradeFinance", ("letter of credit", "bank guarantee", "guarantee")),
    ("TreasuryFX", ("spot fx", "forward fx", "fx swaps", "swap", "forward", "spot")),
    ("International", ("international", "multi-currency", "foreign currency account")),
    ("PaymentsOps", ("swift", "draft", "remittance", "standing orders", "direct debits")),
    ("Digital", ("internet banking", "withme", "app", "e-statements", "alerts")),
    ("Insurance", ("insurance",)),
    ("Deposits", ("deposit", "term deposit", "fixed deposit")),
    ("Retail", ("savings account", "current account", "account")),
    ("SME", ("sme", "project finance", "business", "overdraft", "equipment")),
    ("PrivateBanking", ("private banking",)),
]

# Baseline funnel probabilities by category
# (Typical email channel priors; will be transformed by quality)
BASELINES: Dict[Category, Dict[str, float]] = {
    "Retail":        dict(p_open=0.25, p_click=0.08, p_unsub=0.020, p_comp=0.003, p_signup=0.060),
    "Deposits":      dict(p_open=0.28, p_click=0.09, p_unsub=0.018, p_comp=0.003, p_signup=0.065),
    "Loans":         dict(p_open=0.30, p_click=0.10, p_unsub=0.022, p_comp=0.003, p_signup=0.070),
    "Mortgages":     dict(p_open=0.35, p_click=0.12, p_unsub=0.015, p_comp=0.002, p_signup=0.080),
    "Cards":         dict(p_open=0.30, p_click=0.10, p_unsub=0.025, p_comp=0.004, p_signup=0.070),
    "Leasing":       dict(p_open=0.27, p_click=0.09, p_unsub=0.020, p_comp=0.003, p_signup=0.060),
    "SME":           dict(p_open=0.22, p_click=0.07, p_unsub=0.020, p_comp=0.003, p_signup=0.050),
    "TradeFinance":  dict(p_open=0.20, p_click=0.07, p_unsub=0.018, p_comp=0.003, p_signup=0.045),
    "TreasuryFX":    dict(p_open=0.18, p_click=0.06, p_unsub=0.020, p_comp=0.003, p_signup=0.040),
    "International": dict(p_open=0.24, p_click=0.08, p_unsub=0.020, p_comp=0.003, p_signup=0.050),
    "PaymentsOps":   dict(p_open=0.23, p_click=0.08, p_unsub=0.020, p_comp=0.003, p_signup=0.045),
    "Digital":       dict(p_open=0.40, p_click=0.15, p_unsub=0.016, p_comp=0.002, p_signup=0.100),
    "Insurance":     dict(p_open=0.20, p_click=0.05, p_unsub=0.022, p_comp=0.003, p_signup=0.040),
    "PrivateBanking":dict(p_open=0.26, p_click=0.09, p_unsub=0.015, p_comp=0.002, p_signup=0.060),
}

# Budget ranges by category (MUR, purely synthetic)
BUDGET_RANGES: Dict[Category, Tuple[int, int]] = {
    "Mortgages": (80_000, 200_000),
    "Loans": (40_000, 120_000),
    "Cards": (60_000, 150_000),
    "Digital": (40_000, 100_000),
    "Deposits": (30_000, 90_000),
    "Retail": (25_000, 80_000),
    "Leasing": (30_000, 100_000),
    "SME": (40_000, 120_000),
    "TradeFinance": (25_000, 70_000),
    "TreasuryFX": (25_000, 70_000),
    "International": (30_000, 90_000),
    "PaymentsOps": (20_000, 60_000),
    "Insurance": (20_000, 60_000),
    "PrivateBanking": (50_000, 130_000),
}

QualityTier = Literal["very_good", "very_bad", "average"]

QUALITY_MULTIPLIERS: Dict[QualityTier, Dict[str, float]] = {
    # Multipliers applied to baselines
    "very_good": dict(p_open=1.8, p_click=1.9, p_unsub=0.7, p_comp=0.7, p_signup=1.8),
    "very_bad":  dict(p_open=0.30, p_click=0.30, p_unsub=1.3, p_comp=1.3, p_signup=0.30),
    "average":   dict(p_open=1.00, p_click=1.00, p_unsub=1.00, p_comp=1.00, p_signup=1.00),
}

# ──────────────────────────────────────────────────────────────
# Utility functions
# ──────────────────────────────────────────────────────────────

def random_ipv4() -> str:
    return ".".join(str(random.randint(1, 254)) for _ in range(4))


def normalize_products(raw: Iterable[str]) -> List[str]:
    """Deduplicate, lowercase compare, preserve original casing."""
    seen: set[str] = set()
    out: List[str] = []
    augmented = list(raw) + EXTRA_GENERIC
    for p in augmented:
        key = p.strip().lower()
        if key and key not in seen:
            seen.add(key)
            out.append(p.strip())
    return out


def categorize(product_name: str) -> Category:
    name = product_name.lower()
    for cat, tokens in CATEGORY_RULES:
        if any(t in name for t in tokens):
            return cat
    # Default fallback
    return "Retail"


def choose_budget(category: Category) -> int:
    lo, hi = BUDGET_RANGES[category]
    return random.randint(lo, hi)


def choose_dates() -> Tuple[datetime, datetime]:
    start_earliest = NOW - timedelta(days=START_RANGE_DAYS[0])
    start_latest = NOW - timedelta(days=START_RANGE_DAYS[1])
    start_date = start_earliest + (start_latest - start_earliest) * random.random()
    # duration between MIN and MAX, but not beyond NOW
    max_dur = min(timedelta(days=MAX_DURATION_DAYS), NOW - start_date)
    min_dur = timedelta(days=MIN_DURATION_DAYS)
    duration = min_dur + (max_dur - min_dur) * random.random()
    end_date = start_date + duration
    return start_date, end_date


def cycle_quality(idx: int) -> QualityTier:
    """
    Enforce presence of very good and very bad campaigns:
    pattern: good, bad, avg, avg, good, bad, avg, ...
    """
    r = idx % 5
    if r == 0:
        return "very_good"
    if r == 1:
        return "very_bad"
    return "average"


def apply_quality(baseline: Dict[str, float], tier: QualityTier) -> Dict[str, float]:
    m = QUALITY_MULTIPLIERS[tier]
    q = {k: max(0.0, min(1.0, baseline[k] * m[k])) for k in baseline.keys()}

    # Small random jitter within ±10% (except complaints)
    for k in ("p_open", "p_click", "p_unsub", "p_signup"):
        jitter = 1.0 + random.uniform(-0.10, 0.10)
        q[k] = max(0.0, min(1.0, q[k] * jitter))

    # Logical constraints
    q["p_click"] = min(q["p_click"], 0.95 * q["p_open"])
    q["p_comp"] = min(q["p_comp"], 0.02)

    return q


# ──────────────────────────────────────────────────────────────
# Data generation (campaigns, sends, events, signups)
# ──────────────────────────────────────────────────────────────

def generate_campaigns(products: List[str]) -> pd.DataFrame:
    rows: List[Dict[str, object]] = []
    for i, prod in enumerate(products, start=1):
        cat = categorize(prod)
        start_date, end_date = choose_dates()
        budget = choose_budget(cat)
        cid = str(uuid.uuid4())

        # Campaign name includes the product (schema unchanged)
        name = f"{prod}"

        rows.append(
            {
                "campaign_id": cid,
                "name": name,
                "start_date": start_date,
                "end_date": end_date,
                "budget": budget,
                "_category": cat,          # internal (not written to DB)
                "_quality": cycle_quality(i - 1),  # internal
            }
        )
    return pd.DataFrame(rows)


def assign_qualities(camps: pd.DataFrame) -> Dict[str, Dict[str, float]]:
    qmap: Dict[str, Dict[str, float]] = {}
    for _, r in camps.iterrows():
        cid = str(r["campaign_id"])
        cat: Category = r["_category"]
        tier: QualityTier = r["_quality"]
        baseline = BASELINES[cat]
        qmap[cid] = apply_quality(baseline, tier)
    return qmap


def generate_send_log(camps: pd.DataFrame) -> pd.DataFrame:
    recs: List[Dict[str, object]] = []
    for _, c in camps.iterrows():
        cid = str(c["campaign_id"])
        # Each campaign sends one email to each client (A/B)
        for client in CLIENTS:
            send_ts = c["start_date"] + (c["end_date"] - c["start_date"]) * random.random()
            recs.append(
                {
                    "campaign_id": cid,
                    "msg_id": str(uuid.uuid4()),
                    "recipient": EMAIL_TMPL.format(client=client),
                    "variant": random.choice(["A", "B"]),
                    "send_ts": send_ts,
                }
            )
    return pd.DataFrame(recs)


def generate_event_log(
    sends: pd.DataFrame, qmap: Dict[str, Dict[str, float]], camps: pd.DataFrame
) -> pd.DataFrame:
    camp_info = camps.set_index("campaign_id").to_dict("index")
    recs: List[Tuple[str, str, str, datetime]] = []

    for _, s in sends.iterrows():
        cid = str(s["campaign_id"])
        msg = str(s["msg_id"])
        ts: datetime = s["send_ts"]
        q = qmap[cid]
        start = camp_info[cid]["start_date"]
        end = camp_info[cid]["end_date"]
        span_days = max(1, int((end - ts).days))

        # send
        recs.append((cid, msg, "send", ts))

        # open vs deleted_or_spam
        if random.random() < q["p_open"]:
            ot = ts + timedelta(hours=random.uniform(0.5, 72.0))
            recs.append((cid, msg, "open", ot))
            # click conditional on open
            if random.random() < q["p_click"]:
                ct = ot + timedelta(hours=random.uniform(0.1, 48.0))
                recs.append((cid, msg, "click", ct))
        else:
            dt = ts + timedelta(days=3, hours=random.uniform(0, 12))
            recs.append((cid, msg, "deleted_or_spam", dt))

        # unsubscribe
        if random.random() < q["p_unsub"]:
            ut = ts + timedelta(days=random.uniform(1, span_days))
            recs.append((cid, msg, "unsubscribe", ut))

        # complaint
        if random.random() < q["p_comp"]:
            ct = ts + timedelta(days=random.uniform(1, span_days))
            recs.append((cid, msg, "complaint", ct))

    return pd.DataFrame(recs, columns=["campaign_id", "msg_id", "event_type", "event_ts"])


def generate_user_signup(camps: pd.DataFrame, qmap: Dict[str, Dict[str, float]]) -> pd.DataFrame:
    recs: List[Dict[str, object]] = []
    for _, c in camps.iterrows():
        cid = str(c["campaign_id"])
        start: datetime = c["start_date"]
        end: datetime = c["end_date"]
        span_days = max(1, int((end - start).days))
        p_signup = qmap[cid]["p_signup"]

        # Expected signups ~ Binomial(NUM_CLIENTS, p_signup), but we keep a floor of 1 if p>0
        n = max(1 if p_signup > 0 else 0, int(round(NUM_CLIENTS * p_signup)))
        chosen = random.sample(CLIENTS, k=min(n, len(CLIENTS))) if n > 0 else []

        for client in chosen:
            st = start + timedelta(days=random.uniform(1, span_days))
            recs.append(
                {
                    "signup_id": str(uuid.uuid4()),
                    "campaign_id": cid,
                    "client_name": client,
                    "email": EMAIL_TMPL.format(client=client),
                    "signup_ts": st,
                }
            )
    return pd.DataFrame(recs)


# ──────────────────────────────────────────────────────────────
# I/O helpers
# ──────────────────────────────────────────────────────────────

def write_sqlite(path: str, table: str, df: pd.DataFrame, schema: str) -> None:
    conn = sqlite3.connect(path, detect_types=sqlite3.PARSE_DECLTYPES)
    cur = conn.cursor()
    cur.executescript(f"DROP TABLE IF EXISTS {table};\n{schema}")
    df.to_sql(table, conn, if_exists="append", index=False)
    conn.close()


# ──────────────────────────────────────────────────────────────
# Build pipeline
# ──────────────────────────────────────────────────────────────

def build_all() -> None:
    # Products
    products = normalize_products(RAW_PRODUCTS)

    # Optional sampling if MAX_CAMPAIGNS is set
    if MAX_CAMPAIGNS is not None and MAX_CAMPAIGNS < len(products):
        products = random.sample(products, k=MAX_CAMPAIGNS)

    # 1) campaigns.db
    camps_df = generate_campaigns(products)
    qualities = assign_qualities(camps_df)
    signup_df = generate_user_signup(camps_df, qualities)
    write_sqlite("campaigns.db", "campaigns", camps_df[["campaign_id", "name", "start_date", "end_date", "budget"]], SCHEMA_CAMPAIGNS)
    write_sqlite("campaigns.db", "user_signup", signup_df, SCHEMA_SIGNUP)

    # 2) email_map.db
    sends_df = generate_send_log(camps_df)
    write_sqlite("email_map.db", "email_map", sends_df, SCHEMA_EMAIL_MAP)

    # 3) email_events.db → event_log + events
    ev_df = generate_event_log(sends_df, qualities, camps_df)
    write_sqlite("email_events.db", "event_log", ev_df, SCHEMA_EVENT_LOG)

    # Build `events` by joining campaign name and adding client IP
    conn_e = sqlite3.connect("email_events.db", detect_types=sqlite3.PARSE_DECLTYPES)
    elog = pd.read_sql("SELECT * FROM event_log", conn_e)
    conn_e.close()

    camps_conn = sqlite3.connect("campaigns.db")
    camps_min = pd.read_sql("SELECT campaign_id, name FROM campaigns", camps_conn)
    camps_conn.close()

    evt = elog.merge(camps_min, on="campaign_id", how="left")
    evt["client_ip"] = [random_ipv4() for _ in range(len(evt))]
    evt = evt.rename(columns={"event_ts": "ts", "name": "campaign"})
    evt = evt[["msg_id", "event_type", "client_ip", "ts", "campaign"]]
    write_sqlite("email_events.db", "events", evt, SCHEMA_EVENTS)

    print("▶️ Databases created: campaigns.db, email_map.db, email_events.db")


if __name__ == "__main__":
    build_all()


C:\Users\jvvce\AppData\Local\Temp\ipykernel_23768\2219714933.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  NOW: datetime = datetime.utcnow()


▶️ Databases created: campaigns.db, email_map.db, email_events.db
